In [5]:
from flask import Flask, url_for, request, jsonify
from flask_cors import CORS, cross_origin

import json
import requests

In [6]:
NAVER_USER_DATA_SAVED_FILE = "naver/account"

In [ ]:
import pickle

naver_user_info = dict({
    "smtp_user_id": "주소@naver.com",
    "smtp_user_pw": "비밀번호"
})

f = open(NAVER_USER_DATA_SAVED_FILE, 'wb')
pickle.dump(naver_user_info, f)
f.close()

In [7]:
import pickle

f = open(NAVER_USER_DATA_SAVED_FILE, 'rb')
naver_user_info = pickle.load(f)
f.close()

In [8]:
from email.mime.multipart import MIMEMultipart
from email.mime.image import MIMEImage
from email.mime.audio import MIMEAudio
from email.mime.text import MIMEText
from email.mime.base import MIMEBase
from email import encoders

In [9]:
import smtplib

def send_email(smtp_info, naver_user_info, msg):
    with smtplib.SMTP(smtp_info['smtp_server'], smtp_info["smtp_port"]) as server:
        server.starttls()
        
        server.login(naver_user_info['smtp_user_id'], naver_user_info['smtp_user_pw'])
        
        print(msg.as_string())
        res = server.sendmail(msg["from"], msg["to"], msg.as_string())
        
        if not res:
            print("이메일 전송 성공!")
        else:
            print(res)

In [10]:
def make_multipart(msg_dict):
    multi = MIMEMultipart(_subtype = 'mixed')
    
    for key, value in msg_dict.items():
        if key == 'image':
            with open(value['filename'], 'rb') as fp:
                msg = MIMEImage(fp.read(), _subtype = value['subtype'])
                
        msg.add_header('Content-Disposition', 'attachment', filename = os.path.basename(value['filename']))
        
        multi.attach(msg)
        
    return multi

In [11]:
import os

In [ ]:
app = Flask(__name__)
CORS(app)

@app.route('/test-email', methods=['GET'])
def test_email():
    smtp_info = dict({
        "smtp_server": "smtp.naver.com",
        "smtp_port": 587
    })
    
    msg_dict = {
        'image': {
            'maintype': 'image',
            'subtype': 'jpg',
        }
    }
    
    title = "잔망스러운"
    content = "루피"
    sender = naver_user_info['smtp_user_id']
    receiver = naver_user_info['smtp_user_id']
    
    msg = MIMEText(_text = content, _charset = 'utf-8')
    msg_dict['image']['filename'] = 'image/lupi.jpg'
    
    multi = make_multipart(msg_dict)
    
    multi['Subject'] = title
    multi['From'] = sender
    multi['To'] = receiver
    multi.attach(msg)
    
    send_email(smtp_info, naver_user_info, multi)
    
    return jsonify("email send 완료!")

if __name__ == "__main__":
    app.run()

In [2]:
import datetime

In [3]:
KAKAO_TOKEN_SAVED_FILE = "kakao_msg/kakao_token.json"
NAVER_USER_DATA_SAVED_FILE = "naver/account"

In [12]:
def load_tokens(file):
    with open(file) as fp:
        token = json.load(fp)
        
    return token

token = load_tokens(KAKAO_TOKEN_SAVED_FILE)

In [15]:
app = Flask(__name__)
CORS(app)

@app.route('/order-test', methods=['GET'])
def test_email():
    token = load_tokens(KAKAO_TOKEN_SAVED_FILE)
    f = open(NAVER_USER_DATA_SAVED_FILE, 'rb')
    naver_user_info = pickle.load(f)
    f.close()
    
    url = "https://kapi.kakao.com/v2/api/talk/memo/default/send"
    
    headers = {
        "Authorization": "Bearer " + token['access_token']
    }
    
    msg = "호갱님! 주문이 완료되었습니다!!!\n앞으로도 호갱님과 우주 끝까지 함께 하겠습니다!\n구매자: id, 결제금액: 1000000"
    
    data = {
        "template_object": json.dumps({
            "object_type": "text",
            "text": msg,
            "link": {
                "web_url": "https://www.google.com/search?q=%ED%98%B8%EA%B0%B1%EB%8B%98+%EA%B0%90%EC%82%AC%ED%95%A9%EB%8B%88%EB%8B%A4&rlz=1C1CHBD_koKR1024KR1024&sxsrf=ALiCzsYQp6KdjMMGfV-4Be-UUr91UTQtXA:1665045205770&source=lnms&tbm=isch&sa=X&ved=2ahUKEwjsmpngmMv6AhWSMN4KHS60A5YQ_AUoAXoECAEQAw&biw=1449&bih=886&dpr=1#imgrc=LmOrIRssQ7Dc3M"
            },
            "button_title": "확인해봐~"
        })
    }
    
    response = requests.post(url, headers=headers, data=data)
    
    if response.status_code != 200:
        print("에러 발생!")
    
    print("카카오톡 메시지 전송 완료")
    
    smtp_info = dict({
        "smtp_server": "smtp.naver.com",
        "smtp_port": 587
    })
    
    msg_dict = {
        'image': {
            'maintype': 'image',
            'subtype': 'jpg',
        }
    }
    
    title = "xxx 에서 구매한 결제 내역입니다!"
    content = "xxx 에서 yyy를 구매하였습니다!\n앞으로도 저희 xxx를 많이 이용해주시길 바랍니다!\n 감사합니다!"
    sender = naver_user_info['smtp_user_id']
    receiver = naver_user_info['smtp_user_id']
    
    msg = MIMEText(_text = content, _charset = 'utf-8')
    msg_dict['image']['filename'] = 'image/lupi.jpg'
    
    multi = make_multipart(msg_dict)
    
    multi['Subject'] = title
    multi['From'] = sender
    multi['To'] = receiver
    multi.attach(msg)
    
    send_email(smtp_info, naver_user_info, multi)
    
    return jsonify("email 및 카톡 메시지 전송 완료!")

if __name__ == "__main__":
    app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


에러 발생!
카카오톡 메시지 전송 완료
Content-Type: multipart/mixed; boundary="===============0997348351832956313=="
MIME-Version: 1.0
Subject: =?utf-8?b?eHh4IOyXkOyEnCDqtazrp6TtlZwg6rKw7KCcIOuCtOyXreyeheuLiOuLpCE=?=
From: kookiki91@naver.com
To: kookiki91@naver.com

--===============0997348351832956313==
Content-Type: image/jpg
MIME-Version: 1.0
Content-Transfer-Encoding: base64
Content-Disposition: attachment; filename="lupi.jpg"

...
AAAAAAAAAAAf/9k=

--===============0997348351832956313==
Content-Type: text/plain; charset="utf-8"
MIME-Version: 1.0
Content-Transfer-Encoding: base64

eHh4IOyXkOyEnCB5eXnrpbwg6rWs66ek7ZWY7JiA7Iq164uI64ukIQrslZ7snLzroZzrj4Qg7KCA
7Z2sIHh4eOulvCDrp47snbQg7J207Jqp7ZW07KO87Iuc6ri4IOuwlOuejeuLiOuLpCEKIOqwkOyC
rO2VqeuLiOuLpCE=

--===============0997348351832956313==--

